In [1]:
import pandas as pd
import numpy as np

### Preprocessing for all possible TABLES

In [2]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [3]:
len(df["ID_FINCA"].unique())

1231

In [4]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [5]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [6]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_14836\3219566741.py:2: FutureWarning: ['SUPERFICIE', 'PRODUCCION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [7]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [8]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [9]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [10]:
df.loc[df["MODO"]=="1", "MODO"]  = 0
df.loc[df["MODO"]=="2", "MODO"]  = 1

In [11]:
df = df.drop(index=df[df["CAMPAÑA"] == "22"].index)

### 1 - Table without SUPERFICIE

In [185]:
df_temporal = df.drop(axis = 1, columns = ["SUPERFICIE"])

In [186]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)
encoder = ce.OrdinalEncoder(cols = ["CAMPAÑA"])
df_encoded = encoder.fit_transform(df_temporal)

In [169]:
df_temporal.to_csv("D_T_1.csv")
df_encoded.to_csv("D_T_1_encoded.csv")

### 2 - Table without CAMPAÑA and SUPERFICIE

In [187]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA", "SUPERFICIE"])

In [188]:
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

In [172]:
df_temporal.to_csv("D_T_2.csv")
df_encoded.to_csv("D_T_encoded_2.csv")

### 3 - Table without CAMPAÑA

In [12]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA"])

In [14]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [15]:
train = df_encoded[(df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]

In [16]:
X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

In [17]:
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

normalizer = StandardScaler()
norm_X = normalizer.fit_transform(X)
norm_to_predict = normalizer.transform(to_predict)

In [247]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor


model = {"model_name": "SVC", "model": MLPRegressor(), "params": {'hidden_layer_sizes':[(100),(1000,1000,1000)],
                                                                  'learning_rate':["constant", "adaptive"],}}

grid_search = GridSearchCV(model["model"], model["params"], scoring="r2")
grid_search.fit(norm_X, y)

C:\Users\xavid\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


GridSearchCV(estimator=MLPRegressor(),
             param_grid={'hidden_layer_sizes': [100, (100, 100, 100)],
                         'learning_rate': ['constant', 'adaptive']},
             scoring='r2')

In [248]:
print(grid_search.best_params_)
df = pd.DataFrame(grid_search.cv_results_)
df

{'hidden_layer_sizes': (100, 100, 100), 'learning_rate': 'adaptive'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,9.451171,3.131344,0.005322,0.000376,100,constant,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.774989,0.799511,0.870512,0.590862,0.665164,0.740207,0.099641,4
1,9.308895,1.867798,0.004107,0.000587,100,adaptive,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.784408,0.759720,0.861517,0.754148,0.586562,0.749271,0.089934,3
2,8.366059,0.976483,0.005138,0.000371,"(100, 100, 100)",constant,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.891863,0.824670,0.908488,0.864096,0.617077,0.821239,0.105953,2
3,8.098244,1.237529,0.004801,0.000393,"(100, 100, 100)",adaptive,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.821207,0.866981,0.892721,0.871692,0.829611,0.856443,0.026912,1


In [19]:
from sklearn.neural_network import MLPRegressor
modelMLPR = MLPRegressor(hidden_layer_sizes=(1000, 1000, 1000), learning_rate="adaptive")
modelMLPR.fit(norm_X, y)
predicted = modelMLPR.predict(norm_to_predict)

C:\Users\xavid\AppData\Local\Temp\ipykernel_14836\1973200238.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["SUPERFICIE"] = predicted


In [39]:
final_predict = df[~((df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0))]
final_predict["SUPERFICIE"] = predicted
final_train = df[(df["PRODUCCION"] != 0) & (df["SUPERFICIE"] != 0)]

C:\Users\xavid\AppData\Local\Temp\ipykernel_14836\193630758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_predict["SUPERFICIE"] = predicted


In [40]:
final_df = pd.concat([final_predict, final_train])

In [46]:
final_df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,76953,515,4,660,26,1,0,1,1.278273,22215.0,660.000000,0.00000
1,14,84318,515,4,660,26,1,0,1,1.278273,22215.0,660.000000,0.00000
2,14,85579,340,4,520,32,1,0,1,1.824001,20978.0,520.000000,0.00000
3,14,69671,340,4,520,32,1,0,1,2.420002,40722.0,520.000000,0.00000
4,14,14001,852,14,650-660,81,0,0,1,2.515576,14126.0,654.501718,9.19244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8521,21,37461,239,6,700,52,1,0,1,3.680000,28160.1,700.000000,0.00000
8522,21,58769,239,6,700,32,1,0,1,4.250000,41310.0,700.000000,0.00000
8523,21,58769,239,6,700,59,1,0,1,4.160000,45420.0,700.000000,0.00000
8524,21,88928,239,6,700,40,1,0,1,4.750000,56140.0,700.000000,0.00000


In [49]:
final_df.to_csv("first_model/datasets/D_T_3.csv")

In [56]:
df[df["ID_FINCA"]=="3579"]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
615,14,3579,449,12,550-700,32,1,0,1,0.00000,392.500,550.0,150.0
1772,15,3579,449,12,550-700,32,1,0,1,0.00000,469.382,550.0,150.0
2873,16,3579,449,12,550-700,32,1,0,1,0.00000,248.418,550.0,150.0
3925,17,3579,449,12,550-700,32,1,0,1,0.00000,504.680,550.0,150.0
4993,18,3579,449,12,550-700,32,1,0,1,0.00000,426.684,550.0,150.0
6063,19,3579,449,12,550-700,32,1,0,1,0.00000,288.898,550.0,150.0
7090,20,3579,449,12,550-700,32,1,0,1,0.00072,83.433,550.0,150.0
